In [15]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from numpy  import array
from scipy import stats
from gensim.models.callbacks import CallbackAny2Vec

# PREPROCESSING

In [16]:
corpus_filename = '../data/extracted_emoji_sequences.txt'

In [17]:
corpus = open(corpus_filename).read().splitlines()


In [18]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

In [19]:
def onlyEmojiSequences(tokens):
    threshold_emojis = [x for x in tokens if len(x) > 1]
    return threshold_emojis

In [20]:
tokenized_corpus = tokenize_corpus(corpus)
emojiSequences = onlyEmojiSequences(tokenized_corpus)

In [21]:
token_count = sum([len(sentence) for sentence in emojiSequences])
print("The corpus contains {0:,} emoji points.".format(token_count))

The corpus contains 610,256 emoji points.


# SETTING VARIABLES

In [22]:
# Dimensionality of the resulting word vectors.
num_features = 300

# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 2

# Downsample setting for frequent words.
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
# remove later
seed = 1

emoji2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    def __init__(self):
        self.epoch = 0
    def on_epoch_end(self, model):
        print("Epoch #{}  Loss-Value: {}".format(self.epoch, emoji2vec.get_latest_training_loss()))
        self.epoch += 1
        
epoch_logger = EpochLogger()

emoji2vec.build_vocab(emojiSequences, progress_per=10000000)

# TRAINING

In [23]:
emoji2vec.train(emojiSequences, total_examples=emoji2vec.corpus_count, epochs = 50, compute_loss=True, callbacks=[epoch_logger])

Epoch #0  Loss-Value: 105278.6640625
Epoch #1  Loss-Value: 169150.765625
Epoch #2  Loss-Value: 236787.046875
Epoch #3  Loss-Value: 291833.6875
Epoch #4  Loss-Value: 366358.71875
Epoch #5  Loss-Value: 431451.09375
Epoch #6  Loss-Value: 483302.0
Epoch #7  Loss-Value: 535496.0625
Epoch #8  Loss-Value: 586110.25
Epoch #9  Loss-Value: 638934.3125
Epoch #10  Loss-Value: 698479.8125
Epoch #11  Loss-Value: 747007.3125
Epoch #12  Loss-Value: 793715.875
Epoch #13  Loss-Value: 854693.4375
Epoch #14  Loss-Value: 897203.0625
Epoch #15  Loss-Value: 954439.5625
Epoch #16  Loss-Value: 1004379.875
Epoch #17  Loss-Value: 1063259.875
Epoch #18  Loss-Value: 1114722.75
Epoch #19  Loss-Value: 1180580.375
Epoch #20  Loss-Value: 1236854.625
Epoch #21  Loss-Value: 1294300.875
Epoch #22  Loss-Value: 1338833.875
Epoch #23  Loss-Value: 1387425.0
Epoch #24  Loss-Value: 1442999.875
Epoch #25  Loss-Value: 1496312.625
Epoch #26  Loss-Value: 1545001.25
Epoch #27  Loss-Value: 1599247.5
Epoch #28  Loss-Value: 1664913.75

(19445935, 30512800)

In [12]:
if not os.path.exists("trained"):
    os.makedirs("trained")

In [13]:
emoji2vec.save(os.path.join("trained", "2nd.w2v"))

# EVALUATING TRAINED MODEL

In [24]:
emoji2vec = w2v.Word2Vec.load(os.path.join("trained", "2nd.w2v"))

In [25]:
# extracts the 508 Annotator Results as the Gold-Standard
corpus_filename = '../data/EmoSim508.json'
corpus = open(corpus_filename).read()
annotator_similarity_score_508 = list(array(re.findall('(?<=_Annotator_Agreement": )(.*?)(?=\})', corpus)))

# glyph_pairs_1016
unicode_pairs_1016 = re.findall('(?<=unicodelong": "\\\)(.*?)(?=")', corpus)    
glyph_pairs_1016 = [codecs.decode(unicode_pairs_1016[x].replace(str('\\\\'),str('\\')).replace('_',''), 'unicode_escape') for x in range(len(unicode_pairs_1016))]

# computation of Cosine Similarity
goldstandard = []
selftrained = []
for x in range(len(annotator_similarity_score_508)):
    cosineSimilarity = None
    
    emoji1 = glyph_pairs_1016.pop(0)
    emoji2 = glyph_pairs_1016.pop(0)
    
    try:
        cosineSimilarity = emoji2vec.wv.similarity(emoji1, emoji2)
    except:
        print('the cosine similarity between ' + emoji1 + ' and ' + emoji2 + ' could not be computed.')
    
    if(cosineSimilarity is not None):
        selftrained.append(cosineSimilarity)
        goldstandard.append(annotator_similarity_score_508.pop(0))

# computation of SPEARRANK CORRELATION COEFFICIENT
spearmanRank = stats.spearmanr(goldstandard, selftrained)

print('Der Spearman Rank Correlation Coefficient is {}'.format(spearmanRank))

the cosine similarity between 🇬🇧 and 🇺🇸 could not be computed.
the cosine similarity between 🏅 and 🇺🇸 could not be computed.
the cosine similarity between 🇺🇸 and ❤ could not be computed.
the cosine similarity between 🇺🇸 and 💥 could not be computed.
the cosine similarity between 🎤 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 📲 could not be computed.
the cosine similarity between 👇 and 🇳🇬 could not be computed.
the cosine similarity between 🎧 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 🎶 could not be computed.
the cosine similarity between 👏 and ↪ could not be computed.
Der Spearman Rank Correlation Coefficient is SpearmanrResult(correlation=0.5226083857910726, pvalue=2.9945937436152867e-36)


/Users/smoky/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [26]:
emoji2vec.wv.most_similar('😟')



[('😧', 0.5780831575393677),
 ('😑', 0.5687667727470398),
 ('😖', 0.5309852361679077),
 ('😦', 0.5202800035476685),
 ('😯', 0.4996958374977112),
 ('🤒', 0.47872912883758545),
 ('😣', 0.47099769115448),
 ('😕', 0.470908522605896),
 ('😨', 0.46285319328308105),
 ('🙁', 0.4577151834964752)]

In [33]:
emoji2vec.wv.similarity('🙁', '🤒')

0.5035581218413427

In [34]:
# important check how results changes with different setting!!
# combination of trying out stuff and googling!!